In [21]:
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
import numpy as np
import pandas as pd
from rnn import StochasticLSTM
from utils import DropoutBCELoss

In [22]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score

In [23]:
training_table = pd.read_table("./data/hill-valley/Hill_Valley_without_noise_Training.data", sep=',', dtype=np.float64)
testing_table = pd.read_table("./data/hill-valley/Hill_Valley_without_noise_Testing.data", sep=',', dtype=np.float64)

input_columns = [f"X{i}" for i in range(1, 101)]
label_column = "class"

In [24]:
training_table.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X92,X93,X94,X95,X96,X97,X98,X99,X100,class
0,1317.265789,1315.220951,1312.770581,1309.834252,1306.315588,1302.099102,1297.046401,1290.991646,1283.736109,1275.041652,...,1327.575109,1327.575350,1327.575552,1327.575719,1327.575859,1327.575976,1327.576074,1327.576155,1327.576223,0.0
1,7329.967624,7379.907443,7441.799231,7518.503422,7613.565031,7731.377492,7877.385707,8058.337694,8282.596458,8560.526497,...,7121.300474,7121.300438,7121.300410,7121.300387,7121.300368,7121.300353,7121.300341,7121.300331,7121.300323,1.0
2,809.421410,809.780119,810.207191,810.715653,811.321016,812.041748,812.899834,813.921452,815.137768,816.585886,...,807.545134,807.544181,807.543381,807.542709,807.542144,807.541670,807.541272,807.540937,807.540656,1.0
3,45334.208880,45334.213560,45334.219060,45334.225500,45334.233050,45334.241910,45334.252300,45334.264480,45334.278760,45334.295520,...,47550.921710,47224.457710,46946.072760,46708.686150,46506.259970,46333.645520,46186.452370,46060.936670,45953.905930,1.0
4,1.810359,1.810359,1.810359,1.810359,1.810359,1.810359,1.810359,1.810359,1.810359,1.810359,...,1.790275,1.794794,1.798296,1.801010,1.803114,1.804744,1.806008,1.806987,1.807746,0.0


In [25]:
batch_size = 10
dim = 1
seq_length = 100

In [26]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(training_table[input_columns])

x_train = scaler.transform(training_table[input_columns])
y_train = training_table[label_column].values
x_test = scaler.transform(testing_table[input_columns])
y_test = testing_table[label_column].values

N = len(y_train)

train_dl = torch.utils.data.DataLoader(
        [(x_train[i], y_train[i]) for i in range(len(y_train))],
        batch_size=batch_size,
        num_workers=2,
        shuffle=True
)

def format_input(input_batch):
    return input_batch.transpose(1, 0).reshape(seq_length, -1, dim)

In [27]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.rnn = StochasticLSTM(1, 50, 0.75)
        self.fc1 = nn.Linear(50, 25)
        self.fc2 = nn.Linear(25, 1)
    
    def forward(self, x):
        out, _ = self.rnn(x)
        out = out[-1]
        out = self.fc1(out)
        out = torch.sigmoid(self.fc2(out))
        return out

In [28]:
net = Net().double()

In [29]:
criterion = DropoutBCELoss(1, 1, 0.2)
optimizer = optim.Adam(net.parameters(), lr=0.0001)

In [30]:
for epoch in range(4):
    running_loss = 0.0
    
    for i, (inputs, labels) in enumerate(train_dl):
        inputs = format_input(inputs)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs.flatten(), labels.double(), net.rnn.named_parameters(), N)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.6f' %
                  (epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0

print("Finish training")

[1,    50] loss: 0.703095
[1,   100] loss: 0.698646
[1,   150] loss: 0.699577
[1,   200] loss: 0.700538
[1,   250] loss: 0.699553
[1,   300] loss: 0.710885
[1,   350] loss: 0.698429
[1,   400] loss: 0.700032
[1,   450] loss: 0.710893
[1,   500] loss: 0.700282
[1,   550] loss: 0.702702
[1,   600] loss: 0.702067
[2,    50] loss: 0.699280
[2,   100] loss: 0.701413
[2,   150] loss: 0.701893
[2,   200] loss: 0.703210
[2,   250] loss: 0.704761
[2,   300] loss: 0.702725
[2,   350] loss: 0.700574
[2,   400] loss: 0.700950
[2,   450] loss: 0.704881
[2,   500] loss: 0.700955
[2,   550] loss: 0.702060
[2,   600] loss: 0.700629
[3,    50] loss: 0.703008
[3,   100] loss: 0.702231
[3,   150] loss: 0.701620
[3,   200] loss: 0.702789
[3,   250] loss: 0.702143
[3,   300] loss: 0.701335
[3,   350] loss: 0.698815
[3,   400] loss: 0.704552
[3,   450] loss: 0.699986
[3,   500] loss: 0.703981
[3,   550] loss: 0.701397
[3,   600] loss: 0.702366
[4,    50] loss: 0.701985
[4,   100] loss: 0.700976
[4,   150] l

In [31]:
with torch.no_grad():
    outputs = net(torch.tensor(x_test.reshape(606, 100, 1).swapaxes(1, 0)))
    outputs = outputs.flatten().data.numpy()

In [32]:
roc_auc_score(y_test, outputs)

0.508485476047741